In [1]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install unicodecsv


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


<h1><u>Sprint 3</u> : Programación numérica, dataframes y análisis estadístico </h1> 

<h1><b>Tasca 02 </b></h1>

- Introducción a la librería Pandas, a los Dataframes y los métodos.
- Análisis exploratorio de los datos.

<h2>Ejercicio 1</h2>
<b><u>Instrucción</u></b> Descarga el data set Airlines Delay: Airline on-time statistics and delay causas y cárgalo en un Pandas Dataframe. Explora los datos que contiene, explica brevemente qué variables existen y quedate únicamente con las columnas que consideres relevantes. Justifica tu elección.

In [4]:
import unicodecsv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy import stats
from scipy.stats import norm

pd.options.display.float_format = "{:.0f}".format
np.random.seed(110693)

Retrasos = pd.read_csv("DelayedFlights.csv",index_col=0)

In [5]:
# Imprimir las columnas de la BBDD, las cuales intentaremos describir y analizar para concluir si nos es útil o no cada columna para realizar un análisis interesante.
print("A continuación se puede observar una lista con las columnas que tiene el data set de vuelos retrasados: \n\n"+str(Retrasos.columns)+"\n\nLas variables {'Year','Month','DayofMonth','DayOfweek'} son variables enteras que representas una fecha. \nLo ideal es aprovechar estas columnas para crear una sola columna con la fecha. \n\nLas variables con prefijo CRS tienen que ver con parámetros estimados, \npor ejemplo la hora de salida de un vuelo 'CRSDepTime', etc...\n")
print("La variable 'UniqueCarrier' corresponde al código único de la aerolínea que operó dicho vuelo, \nrecordemos cada observación corresponde a un vuelo distinto.\n")
print("La variable 'ActualElapsedTime' es igual a la suma de 'AirTime' + 'TaxiIn' + 'TaxiOut'")

A continuación se puede observar una lista con las columnas que tiene el data set de vuelos retrasados: 

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

Las variables {'Year','Month','DayofMonth','DayOfweek'} son variables enteras que representas una fecha. 
Lo ideal es aprovechar estas columnas para crear una sola columna con la fecha. 

Las variables con prefijo CRS tienen que ver con parámetros estimados, 
por ejemplo la hora de salida de un vuelo 'CRSDepTime', etc...

La variable 'UniqueCarrier' corresponde al código único de la aerolínea que operó dicho vuelo, 
recordemos cada obser

A continuación creamos algunas nuevas variables, y tomamos una muestra aleatoria de nuestro conjunto de datos para reducir el tamaño del dataset con el que trabajaremos.

In [6]:
# Tomaremos una muestra de 200,000 elementos para trabajar con un dataframe más chico
r=Retrasos.sample(200000,weights="Distance")

# Crear una columna con la fecha del vuelo
r['DepDate'] = pd.to_datetime(r.Year*10000+r.Month*100+r.DayofMonth,format='%Y%m%d')

# Creo una columna llamada 'status', esta columna refleja si el vuelo se considera a tiempo (0), 
# ligeramente retrasado (1), muy retrasado (2), desviado (3), o cancelado (4).
for dataset in r:
    r.loc[r['ArrDelay'] <= 15, 'Status'] = 0
    r.loc[r['ArrDelay'] >= 15, 'Status'] = 1
    r.loc[r['ArrDelay'] >= 60, 'Status'] = 2
    r.loc[r['Diverted'] == 1, 'Status'] = 3
    r.loc[r['Cancelled'] == 1, 'Status'] = 4
    
print(r.head())

         Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
1070181  2008      2          10          7     1441        1420     1705   
3899982  2008      7          23          3     1130        1115     1342   
6305987  2008     11          30          7     2145        2030     2225   
800845   2008      2          23          6     1302        1253     1808   
1497683  2008      3          29          6     1724        1650     2052   

         CRSArrTime UniqueCarrier  FlightNum  ... Cancelled  CancellationCode  \
1070181        1720            AA        902  ...         0                 N   
3899982        1328            US       1679  ...         0                 N   
6305987        2055            MQ       4384  ...         0                 N   
800845         1749            OO       6120  ...         0                 N   
1497683        2020            DL        487  ...         0                 N   

         Diverted  CarrierDelay  WeatherDelay  NAS

Una vez creadas algunas variables adicionales que consideramos interesantes, nos deshacemos de otras que consideremos prescindibles: 

- Year : "Información guardada en la variable DepDate."
- DayofMonth : "Información guardada en la variable DepDate."
- DepTime : "La información relativa a los retrasos del despegue son irrelevantes para nuestro análisis, lo importante se refleja en el retraso final, a la hora de aterrizar."
- DepDelay, ArrTime, CRSArrTime, CRSElapsedTime, Diverted, Cancelled, FlightNum, TailNum : "Todas estas variables no nos sirven para sacar conclusiones, en algunos casos sí, como (Diverted, Cancelled, y posiblemente ArrTime, pero con las variables creadas en el proceso anterior nos aseguramos de obtener la información necesaria de estas y la concentramos en la nueva variable, llamada Status.)

In [7]:
#Cancellation cause
r.loc[r['CancellationCode'] =='A', 'CancellationCode'] = 0
r.loc[r['CancellationCode'] =='B', 'CancellationCode'] = 1
r.loc[r['CancellationCode'] =='C', 'CancellationCode'] = 2

# proceso de deshacernos de las columnas que consideramos innecesarias.
r = r.drop(columns=['Year','DayofMonth','DepTime','DepDelay','ArrTime','CRSArrTime','CRSElapsedTime','Diverted','Cancelled','FlightNum','TailNum']) #Converted to date
# r = r.drop('DayofMonth',1)
# r = r.drop('DepTime',1) 
r.head(5)

,Month,DayOfWeek,CRSDepTime,UniqueCarrier,ActualElapsedTime,AirTime,ArrDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,CancellationCode,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDate,Status
1070181,2,7,1420,AA,144,130,-15,MIA,JFK,1090,4,10,N,NaN,NaN,NaN,NaN,NaN,2008-02-10,0
3899982,7,3,1115,US,132,105,14,BOS,CLT,728,4,23,N,NaN,NaN,NaN,NaN,NaN,2008-07-23,0
6305987,11,7,2030,MQ,100,66,90,SDF,ORD,286,24,10,N,0,24,15,0,51,2008-11-30,2
800845,2,6,1253,OO,186,156,19,DEN,PIT,1290,7,23,N,0,0,0,0,19,2008-02-23,1
1497683,3,6,1650,DL,208,188,32,ATL,SJU,1547,6,14,N,0,3,0,0,29,2008-03-29,1


In [8]:
print('A continuación podemos observar la lista de las variables con las que trabajaremos a partir de los \ncambios anteriores para el dataframe delayedflights.csv : \n\n'+str(r.columns))

A continuación podemos observar la lista de las variables con las que trabajaremos a partir de los 
cambios anteriores para el dataframe delayedflights.csv : 

Index(['Month', 'DayOfWeek', 'CRSDepTime', 'UniqueCarrier',
       'ActualElapsedTime', 'AirTime', 'ArrDelay', 'Origin', 'Dest',
       'Distance', 'TaxiIn', 'TaxiOut', 'CancellationCode', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'DepDate', 'Status'],
      dtype='object')


<h2>Ejercicio 2</h2>
<b><u>Instrucción</u></b> Haz un informe completo del dataset:

- Resume estadísticamente el dataset y las columnas de interés. Haz un análisis estadístico de lo que consideres relevante.
- Encuentra cuántos datos faltantes hay por columna.
- Crea nuevas columnas (velocidad media del vuelo, si ha llegado tarde o no...).
- Haz una tabla de las aerolíneas con mayores retrasos acumulados.
- ¿Cuáles son los vuelos más largos? ¿Y los más atrasados? Busca las rutas más largas y las que acumulen más retrasos.
- Aporta lo que consideres relevante.

In [9]:
# Para resumir estadísticamente el dataset utilizaremos la función describe()
r.describe()

,Month,DayOfWeek,CRSDepTime,ActualElapsedTime,AirTime,ArrDelay,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDate,Status
count,200000,200000,200000,199024,199024,199024,200000,199202,199967,125259,125259,125259,125259,125259,200000,200000
mean,6,4,1458,181,155,41,1160,7,19,20,3,16,0,25,2008-06-19 01:55:58.656000,1
min,1,1,1,17,2,-67,49,0,0,0,0,0,0,0,2008-01-01 00:00:00,0
25%,3,2,1125,116,91,7,617,4,11,0,0,0,0,0,2008-03-19 00:00:00,0
50%,6,4,1500,161,135,23,989,6,15,4,0,2,0,7,2008-06-14 00:00:00,1
75%,9,6,1814,232,205,54,1597,8,21,21,0,16,0,32,2008-09-06 00:00:00,1
max,12,7,2359,750,664,2461,4962,199,342,1510,1297,890,164,1002,2008-12-31 00:00:00,4
std,3,2,433,87,84,58,707,5,15,45,20,34,2,43,NaN,1


In [10]:
# Analizamos la proporción de vuelos por estatus.
print("Primero vemos que de la muestra de 200,000 vuelos \n\nun"+str(format(np.round((len(r[r['Status']==0])/len(r)),10),' %'))+" de los vuelos llegaron a tiempo, \nun"+str(format(np.round((len(r[r['Status']==1])/len(r)),10),' %'))+" de los vuelos estuvieron ligeramente retrasados, \nun"+str(format(np.round((len(r[r['Status']==2])/len(r)),10),' %'))+" de los vuelos estuvieron muy retrasados, \nun"+str(format(np.round((len(r[r['Status']==3])/len(r)),10),' %'))+" de los vuelos fueron desviados, \nun"+str(format(np.round((len(r[r['Status']==4])/len(r)),10),' %'))+" de los vuelos fueron cancelados.\n\nLo que representa "+str(len(r[r['Status']==4]))+" vuelos cancelados, más adelante decidiremos qué hacer con estos vuelos.")

Primero vemos que de la muestra de 200,000 vuelos 

un 36.882500% de los vuelos llegaron a tiempo, 
un 40.109500% de los vuelos estuvieron ligeramente retrasados, 
un 22.520000% de los vuelos estuvieron muy retrasados, 
un 0.464000% de los vuelos fueron desviados, 
un 0.024000% de los vuelos fueron cancelados.

Lo que representa 48 vuelos cancelados, más adelante decidiremos qué hacer con estos vuelos.


Como podemos observar, de los motivos que pueden ocasionar un retraso de un vuelo, vemos que el retraso causado directamente por la aerolínea en términos generales tiene un mayor impacto en el tiempo de retraso, es la variable que presente el retraso máximo de la muestra y el presenta el segundo mayor retraso promedio seguido de LateAircraftDelay.

- Encuentra cuántos datos faltantes hay por columna.

In [11]:
# Analizaré a continuación cuantos datos faltantes tenemos en la muestra elegida.
print("A continuación podemos observar una tabla con la totalidad de datos faltantes por columna en nuestro dataset reducido 'r'\n\n"+str(r.isna().sum()))

A continuación podemos observar una tabla con la totalidad de datos faltantes por columna en nuestro dataset reducido 'r'

Month                    0
DayOfWeek                0
CRSDepTime               0
UniqueCarrier            0
ActualElapsedTime      976
AirTime                976
ArrDelay               976
Origin                   0
Dest                     0
Distance                 0
TaxiIn                 798
TaxiOut                 33
CancellationCode         0
CarrierDelay         74741
WeatherDelay         74741
NASDelay             74741
SecurityDelay        74741
LateAircraftDelay    74741
DepDate                  0
Status                   0
dtype: int64


- Crea nuevas columnas (velocidad media del vuelo, si ha llegado tarde o no...).

In [12]:
# A pesar de haber creado anteriormente algunas columnas ya, crearemos una más, la de velocidad, 
# tener cuidado con las unidades. Ya que la distancia está en millas y el tiempo está en minutos. 
# Pero la velocidad la queremos en millas por hora. 
r['Velocity']=r['Distance']/r['AirTime']*60
r.head(5)

,Month,DayOfWeek,CRSDepTime,UniqueCarrier,ActualElapsedTime,AirTime,ArrDelay,Origin,Dest,Distance,...,TaxiOut,CancellationCode,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDate,Status,Velocity
1070181,2,7,1420,AA,144,130,-15,MIA,JFK,1090,...,10,N,NaN,NaN,NaN,NaN,NaN,2008-02-10,0,503
3899982,7,3,1115,US,132,105,14,BOS,CLT,728,...,23,N,NaN,NaN,NaN,NaN,NaN,2008-07-23,0,416
6305987,11,7,2030,MQ,100,66,90,SDF,ORD,286,...,10,N,0,24,15,0,51,2008-11-30,2,260
800845,2,6,1253,OO,186,156,19,DEN,PIT,1290,...,23,N,0,0,0,0,19,2008-02-23,1,496
1497683,3,6,1650,DL,208,188,32,ATL,SJU,1547,...,14,N,0,3,0,0,29,2008-03-29,1,494


- Haz una tabla de las aerolíneas con mayores retrasos acumulados.

In [16]:
# A continuación vamos a mostrar cuáles son las 5 aerolíneas con el mayor retraso acumulado de vuelos durante el 2008.
print("A continuación observamos la tabla que contiene información de las 5 \naerolíneas que tienen mayor retraso acumulado en el año 2008.")
r.groupby('UniqueCarrier').agg({'ArrDelay':{'sum','min','max'},'Distance':'max'})[('ArrDelay','sum')].sort_values(ascending=False).head(5)

A continuación observamos la tabla que contiene información de las 5 
aerolíneas que tienen mayor retraso acumulado en el año 2008.


UniqueCarrier
AA   1282358
WN    954309
UA    929416
CO    607849
DL    548435
Name: (ArrDelay, sum), dtype: float64

- ¿Cuáles son los vuelos más largos? ¿Y los más atrasados? Busca las rutas más largas y las que acumulen más retrasos.

In [26]:
# A continuación vamos a mostrar cuáles son los 5 vuelos más largos.
print("A continuación observamos una tabla con las rutas más largas, \npodemos observar el aeropuesto de origen y el aeropuesto destino.")
r.groupby(['Origin','Dest']).agg({'ArrDelay':{'sum','min','max'},'Distance':'max'})[('Distance','max')].sort_values(ascending=False).head(6)

A continuación observamos una tabla con las rutas más largas, 
podemos observar el aeropuesto de origen y el aeropuesto destino.


Origin  Dest
EWR     HNL     4962
HNL     EWR     4962
        ATL     4502
ATL     HNL     4502
HNL     ORD     4243
ORD     HNL     4243
Name: (Distance, max), dtype: int64

In [27]:
# A continuación vamos a mostrar cuáles son los 5 vuelos con mayor retraso acumulado.
print("A continuación observamos una tabla con las rutas con mayor retraso acumulado, \npodemos observar el aeropuesto de origen y el aeropuesto destino.")
r.groupby(['Origin','Dest']).agg({'ArrDelay':{'sum','min','max'},'Distance':'max'})[('ArrDelay','sum')].sort_values(ascending=False).head(6)

A continuación observamos una tabla con las rutas con mayor retraso acumulado, 
podemos observar el aeropuesto de origen y el aeropuesto destino.


Origin  Dest
JFK     LAX    38471
SFO     JFK    37425
LAX     JFK    32445
ORD     LAX    32102
JFK     SFO    31293
LAX     ORD    26345
Name: (ArrDelay, sum), dtype: float64

- Aporta lo que consideres relevante.

<h2>Ejercicio 3</h2>
<b><u>Instrucción</u></b> Exporta el dataset limpio con las nuevas columnas a Excel.

In [30]:
r.to_csv("excel_procesado.csv", index=False)